In [1]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [2]:
def read_data(path_to_csv):
    df = pd.read_csv(path_to_csv, header=0, index_col=0)
    
    # select columns by type
    float_cols = df.select_dtypes(include=['float64']).columns
    str_cols = df.select_dtypes(include=['object']).columns
    
    # fill NAs
    df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)
    
    # convert str to bool for true/false values
    mask = df.applymap(type) != bool
    bool_to_str = {True: 1, False: 0}
    df = df.where(mask, df.replace(bool_to_str))
    
    return df

In [9]:
def get_unique_values(df):
    sequence_list = [[elem for elem in row.split(',')] for row in df['sequence']]
    unique_items = set([item for sublist in sequence_list for item in sublist])

    return unique_items

In [26]:
def get_sequence_info(df, unique_items):
    list_dicts = []
    input_target_values = []
    list_already_seen = []

    for row in range(0, len(df)):
        items = [elem for elem in df.at[row, 'sequence'].split(',')]
    
        for position, item in enumerate(items):
            new_row_nr = row + position
            item_dict = {}
            #sequence = df.loc[row, 'sequence']
        
            # specific for position in sequence
            start_coords_col = 'start_coords' + str(position+1)
            item_dict['start_coords.x'] = df.loc[row, str(start_coords_col + '.x')]
            item_dict['start_coords.y'] = df.loc[row, str(start_coords_col + '.y')]
            item_dict['start_coords.z'] = df.loc[row, str(start_coords_col + '.z')]
            
            item_dict['row'] = row
        
            if position == 0:
                input_value = '<start>'
                target_value = item
                input_target_values.append([input_value, target_value])
                
                list_already_seen.append([])
            
            else:
                input_value = items[position-1]
                target_value = item
                input_target_values.append([input_value, target_value])

                list_already_seen.append(items[:position])
                
            list_dicts.append(item_dict)
    
    return list_dicts, input_target_values, list_already_seen

In [11]:
def get_row_info(df):
    dicts_row = {}
    
    for row in range(0, len(df)):
        row_dict = {}
        items = [elem for elem in df.at[row, 'sequence'].split(',')]
        
        for position, item in enumerate(items):
            item_coordinates_x = 'coordinates_' + item + '.x'
            item_coordinates_y = 'coordinates_' + item + '.y'
            item_coordinates_z = 'coordinates_' + item + '.z'
            row_dict[item_coordinates_x] = df.loc[row, item_coordinates_x]
            row_dict[item_coordinates_y] = df.loc[row, item_coordinates_y]
            row_dict[item_coordinates_z] = df.loc[row, item_coordinates_z]
            
            item_containment = item + '.containment'
            item_strong_k = item + '.strong_k'
            item_mid_k = item + '.mid_k'
            item_food_k = item + '.food_k'
            row_dict[item_containment] = df.loc[row, item_containment]
            row_dict[item_food_k] = df.loc[row, item_food_k]
            row_dict[item_strong_k] = df.loc[row, item_strong_k]
            row_dict[item_mid_k] = df.loc[row, item_mid_k]
        
            dicts_row[row] = row_dict
    
    return dicts_row        

In [12]:
def create_singlestep_df(list_dicts, input_target_values, list_already_seen):
    single_step_df = pd.DataFrame(list_dicts)
    single_step_df.insert(loc=0, column='input', value=0)
    single_step_df.insert(loc=1, column='target', value=0)

    for row in range(0, len(input_target_values)):
        single_step_df.loc[row, 'input'] = input_target_values[row][0]
        single_step_df.loc[row, 'target'] = input_target_values[row][1]
        
    for row, elem in enumerate(list_already_seen):
        for item in unique_items:
            single_step_df.loc[row, str(item + '.already_seen')] = 0
            if item in elem:
                single_step_df.loc[row, str(item + '.already_seen')] = 1
                
    for row in range(0, len(single_step_df)):
        for key, values in dicts_row.items():
            if single_step_df.loc[row, 'row'] == key:
                for k,v in values.items():
                    single_step_df.loc[row, k] = v
                    
    single_step_df = single_step_df.drop('row', axis=1)
                    
    return single_step_df

In [28]:
df = read_data('data/task_environments_2022-03-30_transformed.csv')
unique_items = get_unique_values(df)

list_dicts, input_target_values, list_already_seen = get_sequence_info(df, unique_items)
dicts_row = get_row_info(df)

single_step_df = create_singlestep_df(list_dicts, input_target_values, list_already_seen)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [29]:
single_step_df

,input,target,start_coords.x,start_coords.y,start_coords.z,plate.already_seen,blueberries.already_seen,drink.already_seen,plate_small.already_seen,bottle.already_seen,...,food.food_k,food.strong_k,food.mid_k,coordinates_drink.x,coordinates_drink.y,coordinates_drink.z,drink.containment,drink.food_k,drink.strong_k,drink.mid_k
0,<start>,plate,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plate,plate_small,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plate_small,cup,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cup,glass,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glass,knife,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,drink,drink,0.000000,4.000000,1.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1457,drink,spoon,0.000000,4.000000,1.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1458,spoon,cereal,0.000000,2.000000,2.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1459,cereal,glass,0.000000,1.000000,3.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0


In [30]:
date = datetime.today().strftime('%Y-%m-%d')
filename = 'data/single_step_df_ints_' + str(date) + '.csv'

filename

'data/single_step_df_ints_2022-03-30.csv'

In [31]:
single_step_df.to_csv(filename, index=False, header=True)